### 문제 4-2 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용)


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

sk
WD


In [ ]:
import re
from typing import TypedDict, List
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

class CafeState(TypedDict):
    messages: List[BaseMessage]
    query_type: str


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
menu_db = FAISS.load_local(
    "./db/cafe_db",
    embeddings,
    allow_dangerous_deserialization=True
)


def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')

    price_match = re.search(r'₩([\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)

    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }


def classify_query(state: CafeState) -> CafeState:
    user_message = state["messages"][-1].content

    if "가격" in user_message or "얼마" in user_message:
        state["query_type"] = "price"
    elif "추천" in user_message or "뭐 마실까" in user_message:
        state["query_type"] = "recommend"
    elif "메뉴" in user_message or any(m in user_message for m in ["아메리카노", "라떼", "카푸치노"]):
        state["query_type"] = "menu"
    else:
        state["query_type"] = "general"

    return state

def generate_response(state: CafeState) -> CafeState:
    user_message = state["messages"][-1].content

    if state["query_type"] == "price":
        docs = menu_db.similarity_search("메뉴 가격", k=3)
    elif state["query_type"] == "recommend":
        docs = menu_db.similarity_search(user_message, k=3)
        if not docs:
            docs = menu_db.similarity_search("인기 메뉴", k=3)
    elif state["query_type"] == "menu":
        docs = menu_db.similarity_search(user_message, k=3)
    else:
        docs = menu_db.similarity_search(user_message, k=2)

    if docs:
        info_list = [extract_menu_info(d) for d in docs]
        response_text = "\n".join(
            [f"{info['name']} - {info['price']} | {info['description']}" for info in info_list]
        )
    else:
        response_text = "죄송합니다. 관련된 메뉴 정보를 찾을 수 없어요."

    state["messages"].append(AIMessage(content=response_text))
    return state

graph = StateGraph(CafeState)
graph.add_node("classify_query", classify_query)
graph.add_node("generate_response", generate_response)
graph.set_entry_point("classify_query")
graph.add_edge("classify_query", "generate_response")
graph.add_edge("generate_response", END)

app = graph.compile()

if __name__ == "__main__":
    state: CafeState = {
        "messages": [HumanMessage(content="추천해줘")],
        "query_type": "general"
    }
    final_state = app.invoke(state)
    for m in final_state["messages"]:
        print(f"{m.type}: {m.content}")


human: 추천해줘
ai: Unknown - ₩6,000 | 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.
Unknown - ₩7,000 | 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다.
Unknown - ₩4,500 | 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.
